# Data Engineering - Initial Dataset Cleaning

This notebook contains various functions I created to create a dataframe with cleaned comment data. It involved building functions, the function have documentation where neccesary.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import random
import datetime
import re
from collections import Counter
# from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#stuff from nltk
import nltk
from nltk.tokenize import word_tokenize #<- For finding tokens (small divisions) from a large sample of text
from nltk.corpus import stopwords #<- For calling the know stopwords in english (e.g, articles, connectors)
from nltk.corpus import wordnet #<- For calling a lexical database in eglish with meanings, synonyms, antonyms, and more 
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentAnalyzer

########## progress bar
from tqdm.notebook import tqdm, trange
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import time

sid_analyzer = SentimentIntensityAnalyzer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

##### emotions
import text2emotion as te


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
dt_cols = {"author": str, "body": str, "created_utc": str, "score": float}
df = pd.read_csv('df.csv',  usecols=dt_cols, low_memory = False)

In [ ]:
def clean_removed_deleted(df):
    
    """Input = uncleaned reddit data dataframe with a column called 'body'
    
    Output = body column with removed '[deleted]' '[removed]' entries which do not give us
    anything of value for sentiment analysis
    
    '[deleted]' = deleted post by poster
    '[removed]' = deleted by moderators, either auto or manually
    """
    
    # NA's
    df = df.dropna(how='any')
    
    # Removing '[removed]' entries which create noise, account for about 8% of data
    print('dropping [removed] posts')
    removed = df[df.loc[:, 'body'].progress_apply(lambda x: str(x)=="[removed]")]
    index_r = removed.index
    cleaned_r = pd.DataFrame.drop(df, index = index_r)
    
    # Removing '[deleted]' entries to remove noise
    print(' dropping [deleted] posts')
    deleted = cleaned_r[cleaned_r.loc[:,'body'].progress_apply(lambda x: str(x)=="[deleted]")]
    index_d = deleted.index
    cleaned_d = pd.DataFrame.drop(cleaned_r, index = index_d)
    cleaned = cleaned_d

    return cleaned

cleaned_comments  = clean_removed_deleted(df.copy(deep=True))
cleaned_comments

In [ ]:
bot_list = ['AutoModerator', '___alexa___', 'SwapzoneIO', 'ccModBot', 'coinfeeds-bot', 'CryptoMods','[deleted]', '[removed]']

# users = df["author"].value_counts()
# users.head(50)

def remove_bots(df, bot_list, column_name):
    
    """This function is designed to compare authors of comments in dataframe with a predefined list of bot accounts
    
    inputs = dataframe, list of bots, column name (must be a string) in dataframe to compute comparison
    
    output = dataframe with comments by bot accounts removed"""
    is_in = df[~df[column_name].isin(bot_list)]     
    
    return is_in

removed_bots = remove_bots(df=cleaned_comments.copy(deep=True), bot_list = bot_list, column_name = 'author')
removed_bots


In [ ]:
def clean_hyperlinks(df, column_name):
    
    """This function removed all hyperlinks that behin with 'http' and 
    replace with a whitespace and counts number of hyperlinks removed
    
    Input = Dataframe, selected column to remove hyperlinks
    
    Output = Dataframe with removed hyperlinks in text column
    """
#     count = 0
    
#     for comment in df['body']:
#         if (re.subn(r'http\S+', " ", comment)[1]) > 0:
#             count+=1
                 
#     print("removed hyperlinks in {} rows/comments".format(count))
    
    df[column_name] = df[column_name].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
    
    return df

df1 = removed_bots.copy(deep=True)

df2 = clean_hyperlinks(df1, column_name = 'body')
df2

In [ ]:
def get_datetime(df):
    
    """Reddit does not provide a datetime format, comments based on UTC format
    Input = This function takes in Pandas dataframe and expects the presence of a 'created_utc' column
    to convert to Datetime
    
    Output = New column in dataframe called 'datetime' with date present"""

    df['created_utc'] = df['created_utc'].astype(int)
    x = df['created_utc']
    
    datetime = []
    print('creating datetime column')
    for num in tqdm_notebook(x):
        y = pd.Timestamp(num, unit='s', hour=None)
        datetime.append(y)  
    df['date'] = datetime
    df['date'] = pd.to_datetime(df['date']).dt.date
    df
    
    return df

# df1 = removed_bots.copy(deep=True)

datetime = get_datetime(df2)
datetime#.info()

In [ ]:
count = df.groupby([pd.Grouper('date')]).count()#['date'].count()
pd.set_option('display.max_rows', 4000)
count

In [ ]:
def get_sentiment(text:str, analyser, desired_type:str='pos'):
    # Get sentiment from text
    sentiment_score = analyser.polarity_scores(text)
    return sentiment_score[desired_type]

def get_sentiment_scores(df,data_column):
    print('Getting compound sentiment')
    df['Compound Sentiment Score'] = df[data_column].astype(str).progress_apply(lambda x: get_sentiment(x,sid_analyzer,'compound'))
    
#     print('Getting compound sentiment')
#     df['Positive Sentiment Score'] = df[data_column].astype(str).progress_apply(lambda x: get_sentiment(x,sid_analyzer,'pos'))
    
#     print('Getting compound sentiment')
#     df['Negative Sentiment Score'] = df[data_column].astype(str).progress_apply(lambda x: get_sentiment(x,sid_analyzer,'neg'))
    
#     print('Getting compound sentiment')
#     df['Neutral Sentiment Score'] = df[data_column].astype(str).progress_apply(lambda x: get_sentiment(x,sid_analyzer,'neu'))
    return df

# df_small

df = get_sentiment_scores(datetime, 'body')
df

# Dataframe building


In [2]:
# sentiment_df = df.copy(deep=True)
sentiment_df = pd.read_csv('sentiment_df.csv')
sentiment_df

,author,body,created_utc,score,date,Compound Sentiment Score
0,TechnoMagik,I'm not sure how you eliminate spread.. If I a...,1368332818,1.0,2013-05-12,0.9014
1,mytwobitcents,fixed thanks,1368321753,2.0,2013-05-12,0.4404
2,sex_with_a_goat,"The Spanish one is wrong, we don't use 'y' wit...",1368318717,2.0,2013-05-12,-0.4767
3,davidpbrown,"Yes, Russian Trolls are the most obvious answer.",1368298185,2.0,2013-05-11,0.4019
4,bigglejones,"A ""consultant"" asking advice on how to be a co...",1368257850,2.0,2013-05-11,0.0813
...,...,...,...,...,...,...
4553702,Cold_Goose_4242,I want 75k end of week,1612828909,2.0,2021-02-09,0.0772
4553703,damnusernamegotcutof,Have mercy oh crypto gods for I can only get s...,1612828904,2.0,2021-02-09,0.3612
4553704,larrydavid4eyedfuck,"yes thats correct, I own the ethereum wallet t...",1612828903,1.0,2021-02-09,0.7608
4553705,ChocolateMorsels,"This is insanity. Thank you Elon, very cool!",1612828897,3.0,2021-02-09,0.1742


In [3]:

df_positive = sentiment_df.copy(deep=True)
df_negative = sentiment_df.copy(deep=True)

df_positive_drop = df_positive[df_positive['Compound Sentiment Score'] <= 0.25].index
df_positive_drop
df_clean_pos = df_positive.drop(index = df_positive_drop)

df_negative_drop = df_negative[df_negative['Compound Sentiment Score'] >= -0.25].index
df_negative
df_clean_neg = df_negative.drop(index = df_negative_drop)



nonneutral = pd.concat((df_clean_neg, df_clean_pos))
nonneutral

,author,body,created_utc,score,date,Compound Sentiment Score
2,sex_with_a_goat,"The Spanish one is wrong, we don't use 'y' wit...",1368318717,2.0,2013-05-12,-0.4767
5,davidpbrown,Websites get hacked.. news at 11.\n\nDon't kee...,1368255591,3.0,2013-05-11,-0.4019
8,sgodsdogs,it seems there's been a lot of talking heads i...,1368249064,5.0,2013-05-11,-0.5574
23,hyh123,I see. But copycats of the world don't seem to...,1368117303,1.0,2013-05-09,-0.8221
26,AltClubGirls,Sorry that setting was changed by mistake. It ...,1368080817,1.0,2013-05-09,-0.4019
...,...,...,...,...,...,...
4553700,Denaneha,Officially £1 Trillion market cap . CONGRATS C...,1612828918,5.0,2021-02-09,0.8769
4553701,ReformedPony,what coin/coins would you guys put 500-1k into...,1612828916,3.0,2021-02-09,0.4404
4553703,damnusernamegotcutof,Have mercy oh crypto gods for I can only get s...,1612828904,2.0,2021-02-09,0.3612
4553704,larrydavid4eyedfuck,"yes thats correct, I own the ethereum wallet t...",1612828903,1.0,2021-02-09,0.7608


In [ ]:
#neutral dropping
# sentiment_df['Positive Compound'] = sentiment_df['Compound Sentiment Score'] >= 0.25
# sentiment_df['Negative Compound'] = sentiment_df['Compound Sentiment Score'] <= -0.25
# sentiment_df

# neutral = sentiment_df[sentiment_df['Compound Sentiment Score'] == 0.0].index
# neutral
# df_clean = sentiment_df.drop(index = neutral)
# df_clean



In [4]:
nonneutral.to_csv('sentiment_df_0_25.csv', header=True, index=False, columns=list(nonneutral.axes[1]))

In [ ]:
negative = sentiment_df[sentiment_df['Compound Sentiment Score'] <= -0.25].index
negative
positive = sentiment_df[sentiment_df['Compound Sentiment Score'] >= 0.25].index
positive

# to_drop = negative.append(positive)
# to_drop

# df_clean = sentiment_df.drop(index = to_drop)
# df_clean


#  to do: drop all the wrongly binned comments

In [ ]:
df = pd.read_csv('sentiment_df.csv', parse_dates=['date'])
df

In [ ]:
compound_median = df.groupby([pd.Grouper(key='date',freq='D')])['Compound Sentiment Score'].median()
compound_median
print(min(compound_median)),print(max(compound_median))

# df.info()

In [ ]:
for i in compound_median:
    print(i)

In [ ]:
median = pd.DataFrame(compound_median)
for i in range(0,len(median)):
    print([i])
    print(median['Compound Sentiment Score'][i])
    print('-----------------------------------------------------')

In [ ]:
median

In [ ]:
interpolated = compound_median.interpolate(method = 'linear')

In [ ]:
sent_change = []

for i in range(0,len(interpolated)-1):
    change = (interpolated[i+1]-interpolated[i])
    sent_change.append(change)
#     print(interpolated[i+1])

In [ ]:
sent_change

In [ ]:
# sent_change.insert(0,0)
len(sent_change)
median['sentiment change'] = sent_change

In [ ]:
median['Compound Sentiment Score'] = interpolated

In [ ]:
date = median.index
date

In [ ]:
median['date'] = date

In [ ]:
median

In [ ]:
median.to_csv('sentiment_daily.csv', header=True, index=False, columns=list(median.axes[1]))